In [1]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [2]:
def create_data_model():
    return {
        'distance_matrix': [
            [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
            [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
            [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
            [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
            [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
            [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
            [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
            [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
            [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
            [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
            [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
            [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
            [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
            [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
            [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
            [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
            [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        ],
        'num_vehicles': 4,
        'depot': 0,
        'weight': [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8],
        'vehicle_weight_capacities': [15, 15, 15, 15],
        'volume': [0, 1, 1, 2, 1, 2, 4, 3, 3, 1, 2, 1, 2, 2, 2, 3, 2],
        'vehicle_volume_capacities': [6, 8, 10, 12],
    }

In [3]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']),
    data['num_vehicles'],
    data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [4]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [5]:
def weight_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['weight'][from_node]

weight_callback_index = routing.RegisterUnaryTransitCallback(weight_callback)
routing.AddDimensionWithVehicleCapacity(
    weight_callback_index,
    0,  # null capacity slack
    data['vehicle_weight_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Weight')

True

In [6]:
def volume_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['volume'][from_node]

volume_callback_index = routing.RegisterUnaryTransitCallback(volume_callback)
routing.AddDimensionWithVehicleCapacity(
    volume_callback_index,
    0,  # null capacity slack
    data['vehicle_volume_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Volume')

True

In [7]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console"""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    total_volume = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        route_volume = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['weight'][node_index]
            route_volume += data['volume'][node_index]
            plan_output += ' {0} Load({1}kg {2}m^3) -> '.format(node_index, route_load, route_volume)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index,
                index,
                vehicle_id)
        plan_output += ' {0} Load({1}kg {2}m^3)\n'.format(manager.IndexToNode(index), route_load, route_volume)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        plan_output += 'Volume of the route: {}\n'.format(route_volume)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
        total_volume += route_volume
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}m'.format(total_load))
    print('Total volume of all routes: {}m'.format(total_volume))

In [8]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(data, manager, routing, solution)

Objective: 8468
Route for vehicle 0:
 0 Load(0kg 0m^3) ->  1 Load(1kg 1m^3) ->  4 Load(5kg 2m^3) ->  3 Load(7kg 4m^3) ->  16 Load(15kg 6m^3) ->  0 Load(15kg 6m^3)
Distance of the route: 2876m
Load of the route: 15
Volume of the route: 6

Route for vehicle 1:
 0 Load(0kg 0m^3) ->  2 Load(1kg 1m^3) ->  10 Load(3kg 3m^3) ->  14 Load(7kg 5m^3) ->  15 Load(15kg 8m^3) ->  0 Load(15kg 8m^3)
Distance of the route: 3104m
Load of the route: 15
Volume of the route: 8

Route for vehicle 2:
 0 Load(0kg 0m^3) ->  7 Load(8kg 3m^3) ->  13 Load(12kg 5m^3) ->  12 Load(14kg 7m^3) ->  11 Load(15kg 8m^3) ->  0 Load(15kg 8m^3)
Distance of the route: 1324m
Load of the route: 15
Volume of the route: 8

Route for vehicle 3:
 0 Load(0kg 0m^3) ->  9 Load(1kg 1m^3) ->  8 Load(9kg 4m^3) ->  6 Load(13kg 8m^3) ->  5 Load(15kg 10m^3) ->  0 Load(15kg 10m^3)
Distance of the route: 1164m
Load of the route: 15
Volume of the route: 10

Total distance of all routes: 8468m
Total load of all routes: 60m
Total volume of all r